<a href="https://colab.research.google.com/github/Salma-Laanani/AiAgent-MCP/blob/master/FineTuning_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %% [markdown]
# # 🚀 FINE-TUNING EMSI CHATBOT - Llama 3.2 avec LoRA
# ## Dataset: Votre fichier emsi_dataset.json
#
# **Auteur** : Votre nom
# **Date** : Aujourd'hui
# **Objectif** : Fine-tuner Llama 3.2 pour répondre aux questions EMSI

# %%
# Étape 1.1 : Vérifier le GPU (CRUCIAL !)
import torch
print(f"🎯 GPU disponible : {torch.cuda.is_available()}")
print(f"🎯 GPU : {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'AUCUN'}")
print(f"🎯 Mémoire GPU : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} Go" if torch.cuda.is_available() else "")

# Si pas de GPU -> Runtime -> Change runtime type -> GPU

🎯 GPU disponible : True
🎯 GPU : Tesla T4
🎯 Mémoire GPU : 15.83 Go


In [2]:
# %%
# Étape 2 : Installation des dépendances
!pip install -q transformers accelerate peft datasets bitsandbytes
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q sentencepiece protobuf
!pip install -q scikit-learn

print("✅ Toutes les bibliothèques sont installées !")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 20.8 MB/s eta 0:00:00
✅ Toutes les bibliothèques sont installées !


In [3]:
# %%
# Étape 3 : Upload de votre dataset EMSI
from google.colab import files
import json
import pandas as pd

print("📤 UPLOADEZ VOTRE FICHIER emsi_dataset.json")
print("Cliquez sur 'Choose Files' et sélectionnez votre fichier")
print("-" * 50)

uploaded = files.upload()

# Vérification
dataset_filename = list(uploaded.keys())[0]
print(f"✅ Fichier uploadé : {dataset_filename}")
print(f"📦 Taille : {len(uploaded[dataset_filename]) / 1024:.2f} Ko")

# Chargement et vérification
with open(dataset_filename, 'r', encoding='utf-8') as f:
    emsi_data = json.load(f)

print(f"🎯 Nombre d'exemples : {len(emsi_data)}")
print("\n📝 Aperçu du premier exemple :")
print(json.dumps(emsi_data[0], indent=2, ensure_ascii=False))

📤 UPLOADEZ VOTRE FICHIER emsi_dataset.json
Cliquez sur 'Choose Files' et sélectionnez votre fichier
--------------------------------------------------


Saving emsi_dataset.json to emsi_dataset.json
✅ Fichier uploadé : emsi_dataset.json
📦 Taille : 91.16 Ko
🎯 Nombre d'exemples : 200

📝 Aperçu du premier exemple :
{
  "instruction": "Quand a été fondée l'EMSI ?",
  "input": "Document officiel EMSI - Historique",
  "output": "L'École Marocaine des Sciences de l'Ingénieur (EMSI) a été fondée en 1986 au Maroc."
}


In [4]:
# %%
# Étape 4 : Préparation du dataset pour l'entraînement
from datasets import Dataset, DatasetDict
import pandas as pd

print("🧹 Préparation du dataset...")

# Convertir en DataFrame
df = pd.DataFrame(emsi_data)

# Afficher les statistiques
print(f"📊 Statistiques du dataset :")
print(f"   - Total exemples : {len(df)}")
print(f"   - Colonnes : {list(df.columns)}")
print(f"   - Exemples avec contexte : {df['input'].notna().sum()}")

# Vérifier les colonnes nécessaires
required_columns = ['instruction', 'output']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"❌ Colonnes manquantes : {missing_columns}")
    print("Vérifiez votre dataset !")
else:
    print("✅ Dataset correctement formaté")

# %%
# Fonction de formatage pour l'entraînement
def format_instruction(row):
    """Formate les exemples pour l'entraînement"""
    instruction = row['instruction']
    context = row.get('input', '')
    response = row['output']

    if context and str(context).strip():
        return f"### Instruction:\n{instruction}\n\n### Contexte:\n{context}\n\n### Réponse:\n{response}"
    else:
        return f"### Instruction:\n{instruction}\n\n### Réponse:\n{response}"

# Appliquer le formatage
print("🔄 Formatage des exemples...")
df['text'] = df.apply(format_instruction, axis=1)

# Créer le dataset Hugging Face
dataset = Dataset.from_pandas(df[['text']])

# Split train/validation (80/20)
dataset = dataset.train_test_split(test_size=0.2, seed=42, shuffle=True)

print(f"✅ Dataset préparé !")
print(f"   - Train : {len(dataset['train'])} exemples")
print(f"   - Validation : {len(dataset['test'])} exemples")

🧹 Préparation du dataset...
📊 Statistiques du dataset :
   - Total exemples : 200
   - Colonnes : ['instruction', 'input', 'output']
   - Exemples avec contexte : 200
✅ Dataset correctement formaté
🔄 Formatage des exemples...
✅ Dataset préparé !
   - Train : 160 exemples
   - Validation : 40 exemples


In [5]:
# %%
# Étape 5 : Configuration du modèle avec LoRA
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
import torch

print("⚙️ Configuration du modèle...")

# CONFIGURATION 4-BIT (économie de mémoire)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                # Charger en 4-bit
    bnb_4bit_quant_type="nf4",        # Type de quantification
    bnb_4bit_compute_dtype=torch.float16,  # Type de calcul
    bnb_4bit_use_double_quant=True    # Double quantification
)

# CHOIX DU MODÈLE (2 options - choisissez une seule)
MODEL_CHOICE = "llama3.2-1b"  # ⬅️ CHANGEZ ICI

model_options = {
    "llama3.2-1b": "unsloth/llama-3.2-1b",      # Léger (1B) - Recommandé pour Colab gratuit
    "llama3.2-3b": "unsloth/llama-3.2-3b",      # Moyen (3B) - Si vous avez Colab Pro
}

model_name = model_options[MODEL_CHOICE]
print(f"🎯 Modèle sélectionné : {model_name}")

# Chargement du tokenizer
print("📥 Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Important !
tokenizer.padding_side = "right"

print(f"✅ Tokenizer chargé : {tokenizer.__class__.__name__}")

# %%
# Chargement du modèle
print("📥 Chargement du modèle (peut prendre 1-2 minutes)...")

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,  # 4-bit quantization
        device_map="auto",               # Auto-détection GPU
        trust_remote_code=True,          # Nécessaire pour certains modèles
        use_cache=False                  # Désactiver cache pour LoRA
    )

    print("✅ Modèle chargé avec succès !")

except Exception as e:
    print(f"❌ Erreur chargement modèle : {e}")
    print("Essaiez un modèle plus léger (llama3.2-1b)")

⚙️ Configuration du modèle...
🎯 Modèle sélectionné : unsloth/llama-3.2-1b
📥 Chargement du tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

✅ Tokenizer chargé : PreTrainedTokenizerFast
📥 Chargement du modèle (peut prendre 1-2 minutes)...


config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

✅ Modèle chargé avec succès !


In [7]:
# %%
# Étape 7 : Tokenization
print("🔤 Tokenization du dataset...")

def tokenize_function(examples):
    """Tokenize les exemples"""
    # Tokenization avec padding/truncation
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,                # Longueur max (512-1024)
        return_tensors="pt"
    )

    # Pour l'entraînement causal, les labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()

    return tokenized

# Appliquer la tokenization
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]            # Supprimer la colonne texte originale
)

print(f"✅ Tokenization terminée !")
print(f"   - Exemple tokenizé : {len(tokenized_datasets['train'][0]['input_ids'])} tokens")

🔤 Tokenization du dataset...


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

✅ Tokenization terminée !
   - Exemple tokenizé : 512 tokens


In [9]:
# %%
# Étape 8 : Configuration de l'entraînement
from transformers import TrainingArguments, Trainer
import os

# Créer le dossier de sortie
output_dir = "./emsi-llama-lora-model"
os.makedirs(output_dir, exist_ok=True)

print("⚙️ Configuration de l'entraînement...")

training_args = TrainingArguments(
    output_dir=output_dir,

    # Hyperparamètres d'entraînement
    num_train_epochs=5,                # Nombre d'époques (3-10)
    per_device_train_batch_size=2,     # Batch size (1-4 selon GPU)
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,     # Accumulation pour simuler batch plus grand

    # Optimisation
    learning_rate=2e-4,                # Taux d'apprentissage (1e-4 à 5e-4)
    weight_decay=0.01,                 # Régularisation
    warmup_steps=100,                  # Échauffement

    # Sauvegarde et évaluation
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,                # Garder seulement 2 checkpoints

    # Performance
    fp16=True,                         # Mixed precision (économise mémoire)
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Divers
    report_to="none",                  # Pas de reporting externe
    push_to_hub=False,                 # Pas de push sur Hugging Face
    ddp_find_unused_parameters=False,
)

print("✅ Configuration d'entraînement prête !")

⚙️ Configuration de l'entraînement...


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [10]:
# %%
# Étape 8 : Configuration de l'entraînement
from transformers import TrainingArguments, Trainer
import os

# Créer le dossier de sortie
output_dir = "./emsi-llama-lora-model"
os.makedirs(output_dir, exist_ok=True)

print("⚙️ Configuration de l'entraînement...")

training_args = TrainingArguments(
    output_dir=output_dir,

    # Hyperparamètres d'entraînement
    num_train_epochs=5,                # Nombre d'époques (3-10)
    per_device_train_batch_size=2,     # Batch size (1-4 selon GPU)
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,     # Accumulation pour simuler batch plus grand

    # Optimisation
    learning_rate=2e-4,                # Taux d'apprentissage (1e-4 à 5e-4)
    weight_decay=0.01,                 # Régularisation
    warmup_steps=100,                  # Échauffement

    # Sauvegarde et évaluation
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    eval_strategy="steps",             # CHANGÉ: evaluation_strategy → eval_strategy
    save_strategy="steps",
    save_total_limit=2,                # Garder seulement 2 checkpoints

    # Performance
    fp16=True,                         # Mixed precision (économise mémoire)
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Divers
    report_to="none",                  # Pas de reporting externe
    push_to_hub=False,                 # Pas de push sur Hugging Face
    ddp_find_unused_parameters=False,
)

print("✅ Configuration d'entraînement prête !")

⚙️ Configuration de l'entraînement...
✅ Configuration d'entraînement prête !


In [12]:
# %%
# Étape 9 : Entraînement (LA PARTIE IMPORTANTE !)
print("🚀 DÉBUT DE L'ENTRAÎNEMENT...")
print("⏱️  Durée estimée : 30-60 minutes")
print("-" * 50)

# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Démarrer l'entraînement
try:
    training_results = trainer.train()

    print("🎉 ENTRAÎNEMENT TERMINÉ !")
    print(f"📈 Perte finale : {training_results.metrics['train_loss']:.4f}")

except Exception as e:
    print(f"❌ Erreur lors de l'entraînement : {e}")
    print("⚠️  Essayez les solutions suivantes :")
    print("1. Vérifiez que votre dataset est correctement formaté")
    print("2. Diminuez le batch_size (per_device_train_batch_size=1)")
    print("3. Vérifiez la mémoire disponible de votre GPU")

🚀 DÉBUT DE L'ENTRAÎNEMENT...
⏱️  Durée estimée : 30-60 minutes
--------------------------------------------------
❌ Erreur lors de l'entraînement : Attempting to unscale FP16 gradients.
⚠️  Essayez les solutions suivantes :
1. Vérifiez que votre dataset est correctement formaté
2. Diminuez le batch_size (per_device_train_batch_size=1)
3. Vérifiez la mémoire disponible de votre GPU


In [13]:
# %%
# Étape 9 : Entraînement (LA PARTIE IMPORTANTE !)
print("🚀 DÉBUT DE L'ENTRAÎNEMENT...")
print("⏱️  Durée estimée : 30-60 minutes")
print("-" * 50)

# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,  # Ajout important !
)

# Démarrer l'entraînement
try:
    training_results = trainer.train()

    print("🎉 ENTRAÎNEMENT TERMINÉ !")
    print(f"📈 Perte finale : {training_results.metrics['train_loss']:.4f}")

except Exception as e:
    print(f"❌ Erreur lors de l'entraînement : {e}")
    print("\n⚠️  Essayez de diminuer le batch_size (per_device_train_batch_size=1)")

/tmp/ipython-input-759407114.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


🚀 DÉBUT DE L'ENTRAÎNEMENT...
⏱️  Durée estimée : 30-60 minutes
--------------------------------------------------
❌ Erreur lors de l'entraînement : Attempting to unscale FP16 gradients.

⚠️  Essayez de diminuer le batch_size (per_device_train_batch_size=1)


In [14]:
# Étape 8 : Configuration de l'entraînement
from transformers import TrainingArguments, Trainer
import os

# Créer le dossier de sortie
output_dir = "./emsi-llama-lora-model"
os.makedirs(output_dir, exist_ok=True)

print("⚙️ Configuration de l'entraînement...")

training_args = TrainingArguments(
    output_dir=output_dir,

    # Hyperparamètres d'entraînement
    num_train_epochs=5,                # Nombre d'époques (3-10)
    per_device_train_batch_size=1,     # Diminué à 1 pour éviter les erreurs
    per_device_eval_batch_size=1,      # Diminué à 1
    gradient_accumulation_steps=8,     # Augmenté pour compenser le batch réduit

    # Optimisation
    learning_rate=2e-4,                # Taux d'apprentissage (1e-4 à 5e-4)
    weight_decay=0.01,                 # Régularisation
    warmup_ratio=0.1,                  # Échauffement en ratio (au lieu de steps)

    # Sauvegarde et évaluation
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,                # Garder seulement 2 checkpoints

    # Performance - IMPORTANT: Pas de fp16 avec 4-bit
    # fp16=True,                       # RETIRÉ: incompatible avec 4-bit quantization
    # fp16_full_eval=False,            # RETIRÉ aussi
    bf16=False,                        # Désactivé aussi
    gradient_checkpointing=True,       # Économie de mémoire
    optim="paged_adamw_8bit",          # Optimiseur spécial pour 4-bit

    # Chargement du meilleur modèle
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Divers
    report_to="none",                  # Pas de reporting externe
    push_to_hub=False,                 # Pas de push sur Hugging Face
    ddp_find_unused_parameters=False,
)

print("✅ Configuration d'entraînement prête !")

⚙️ Configuration de l'entraînement...
✅ Configuration d'entraînement prête !


In [15]:
# %%
# Étape 9 : Entraînement (LA PARTIE IMPORTANTE !)
print("🚀 DÉBUT DE L'ENTRAÎNEMENT...")
print("⏱️  Durée estimée : 30-60 minutes")
print("-" * 50)

# Créer le Trainer (sans tokenizer pour éviter l'avertissement)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    # tokenizer=tokenizer,  # Retiré comme suggéré par l'avertissement
)

# Démarrer l'entraînement
try:
    training_results = trainer.train()

    print("🎉 ENTRAÎNEMENT TERMINÉ !")
    print(f"📈 Perte finale : {training_results.metrics['train_loss']:.4f}")

    # Sauvegarder le modèle
    print("💾 Sauvegarde du modèle...")
    trainer.save_model()
    print(f"✅ Modèle sauvegardé dans : {output_dir}")

except Exception as e:
    print(f"❌ Erreur lors de l'entraînement : {e}")
    print("\n⚠️  Solutions :")
    print("1. Vérifiez que vous avez assez de mémoire GPU")
    print("2. Diminuez encore le batch_size si nécessaire")
    print("3. Essayez avec moins d'époques (num_train_epochs=3)")
    print("4. Réduisez la longueur max des séquences (max_length=256)")

🚀 DÉBUT DE L'ENTRAÎNEMENT...
⏱️  Durée estimée : 30-60 minutes
--------------------------------------------------
❌ Erreur lors de l'entraînement : Attempting to unscale FP16 gradients.

⚠️  Solutions :
1. Vérifiez que vous avez assez de mémoire GPU
2. Diminuez encore le batch_size si nécessaire
3. Essayez avec moins d'époques (num_train_epochs=3)
4. Réduisez la longueur max des séquences (max_length=256)


In [16]:
# Étape 5 : Configuration du modèle avec LoRA
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
import torch

print("⚙️ Configuration du modèle...")

# CONFIGURATION 4-BIT (économie de mémoire) - MODIFIÉ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                # Charger en 4-bit
    bnb_4bit_quant_type="nf4",        # Type de quantification
    bnb_4bit_compute_dtype=torch.bfloat16,  # CHANGÉ: float16 → bfloat16
    bnb_4bit_use_double_quant=True    # Double quantification
)

# CHOIX DU MODÈLE
MODEL_CHOICE = "llama3.2-1b"

model_options = {
    "llama3.2-1b": "unsloth/llama-3.2-1b",
    "llama3.2-3b": "unsloth/llama-3.2-3b",
}

model_name = model_options[MODEL_CHOICE]
print(f"🎯 Modèle sélectionné : {model_name}")

# Chargement du tokenizer
print("📥 Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ Tokenizer chargé : {tokenizer.__class__.__name__}")

# Chargement du modèle
print("📥 Chargement du modèle (peut prendre 1-2 minutes)...")

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        use_cache=False
    )

    print("✅ Modèle chargé avec succès !")

except Exception as e:
    print(f"❌ Erreur chargement modèle : {e}")
    print("Essaiez un modèle plus léger (llama3.2-1b)")

⚙️ Configuration du modèle...
🎯 Modèle sélectionné : unsloth/llama-3.2-1b
📥 Chargement du tokenizer...
✅ Tokenizer chargé : PreTrainedTokenizerFast
📥 Chargement du modèle (peut prendre 1-2 minutes)...
✅ Modèle chargé avec succès !


In [17]:
# Étape 8 : Configuration de l'entraînement
from transformers import TrainingArguments, Trainer
import os

# Créer le dossier de sortie
output_dir = "./emsi-llama-lora-model"
os.makedirs(output_dir, exist_ok=True)

print("⚙️ Configuration de l'entraînement...")

# D'abord, essayons avec ces paramètres optimisés
training_args = TrainingArguments(
    output_dir=output_dir,

    # Hyperparamètres d'entraînement
    num_train_epochs=3,                # Réduit à 3 époques
    per_device_train_batch_size=1,     # Batch size minimal
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,    # Accumulation pour batch effectif de 16

    # Optimisation
    learning_rate=1e-4,                # Taux d'apprentissage réduit
    weight_decay=0.01,
    warmup_steps=50,                   # Peu d'échauffement

    # Sauvegarde et évaluation
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,

    # Performance - CONFIGURATION SPÉCIALE
    bf16=True,                         # Utilise bfloat16 au lieu de fp16
    gradient_checkpointing=True,       # Économie de mémoire
    optim="paged_adamw_8bit",          # Optimiseur pour 4-bit

    # Chargement du meilleur modèle
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Divers
    report_to="none",
    push_to_hub=False,
    ddp_find_unused_parameters=False,

    # Désactiver les avertissements de précision
    fp16=False,                       # Explicitement désactivé
    tf32=False,                       # Désactivé
)

print("✅ Configuration d'entraînement prête !")

⚙️ Configuration de l'entraînement...
✅ Configuration d'entraînement prête !


In [18]:
# Étape 7 : Préparation des données
print("📊 Préparation des données...")

def tokenize_function(examples):
    # Réduire la longueur maximale pour économiser de la mémoire
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256  # Réduit de 512 à 256
    )

    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

print("🔠 Tokenisation des données...")
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print(f"✅ Données tokenisées !")

📊 Préparation des données...
🔠 Tokenisation des données...


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

✅ Données tokenisées !


In [19]:
# Étape 9 : Entraînement
print("🚀 DÉBUT DE L'ENTRAÎNEMENT...")
print("⏱️  Durée estimée : 15-30 minutes")
print("-" * 50)

# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    # Le tokenizer n'est plus nécessaire ici
)

# Solution alternative si ça ne marche pas
import warnings
warnings.filterwarnings('ignore', message='.*unscale FP16 gradients.*')

# Démarrer l'entraînement
try:
    print("🔄 Démarrage de l'entraînement...")
    training_results = trainer.train()

    print("🎉 ENTRAÎNEMENT TERMINÉ !")
    print(f"📈 Perte finale : {training_results.metrics['train_loss']:.4f}")

    # Sauvegarder
    trainer.save_model()
    print(f"💾 Modèle sauvegardé dans : {output_dir}")

except RuntimeError as e:
    if "unscale FP16 gradients" in str(e):
        print("⚠️  Problème de précision détecté. Tentative de solution...")
        # Réessayer avec une configuration encore plus simple
        training_args.fp16 = False
        training_args.bf16 = False
        training_args.tf32 = False

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
        )

        training_results = trainer.train()
        print("✅ Entraînement réussi après ajustement !")
    else:
        print(f"❌ Autre erreur : {e}")

except Exception as e:
    print(f"❌ Erreur critique : {e}")
    print("Essayez avec un dataset plus petit ou un modèle plus léger.")

🚀 DÉBUT DE L'ENTRAÎNEMENT...
⏱️  Durée estimée : 15-30 minutes
--------------------------------------------------


ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [20]:
# %%
# Étape 5 : Configuration du modèle avec LoRA
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
import torch

print("⚙️ Configuration du modèle...")

# CONFIGURATION 4-BIT (économie de mémoire)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# CHOIX DU MODÈLE
MODEL_CHOICE = "llama3.2-1b"

model_options = {
    "llama3.2-1b": "unsloth/llama-3.2-1b",
    "llama3.2-3b": "unsloth/llama-3.2-3b",
}

model_name = model_options[MODEL_CHOICE]
print(f"🎯 Modèle sélectionné : {model_name}")

# Chargement du tokenizer
print("📥 Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print(f"✅ Tokenizer chargé : {tokenizer.__class__.__name__}")

# %%
# Chargement du modèle
print("📥 Chargement du modèle (peut prendre 1-2 minutes)...")

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        use_cache=False
    )

    print("✅ Modèle chargé avec succès !")

except Exception as e:
    print(f"❌ Erreur chargement modèle : {e}")
    print("Essaiez un modèle plus léger (llama3.2-1b)")

# %%
# Étape 6 : PRÉPARER le modèle pour l'entraînement 4-bit + LoRA
print("🔧 Préparation du modèle pour l'entraînement 4-bit...")

# IMPORTANT: Préparer le modèle pour l'entraînement k-bit
model = prepare_model_for_kbit_training(model)
print("✅ Modèle préparé pour l'entraînement 4-bit")

# Configuration LoRA
print("⚙️ Configuration LoRA...")

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
)

# Appliquer LoRA au modèle
print("🔄 Application de LoRA au modèle...")
model = get_peft_model(model, lora_config)

# Afficher les paramètres entraînables
model.print_trainable_parameters()
print("✅ Configuration LoRA terminée !")

# %%
# Étape 7 : Préparation des données
print("📊 Préparation des données...")

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔠 Tokenisation des données...")
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print(f"✅ Données tokenisées !")
print(f"   Train: {len(tokenized_datasets['train'])} exemples")
print(f"   Test: {len(tokenized_datasets['test'])} exemples")

# %%
# Étape 8 : Configuration de l'entraînement
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

# Créer le dossier de sortie
output_dir = "./emsi-llama-lora-model"
os.makedirs(output_dir, exist_ok=True)

print("⚙️ Configuration de l'entraînement...")

# Data collator pour le langage
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM, pas de masquage
)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=50,
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,

    # Désactiver fp16/bf16 pour éviter les erreurs
    fp16=False,
    bf16=False,

    gradient_checkpointing=True,
    optim="paged_adamw_8bit",

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    push_to_hub=False,
    ddp_find_unused_parameters=False,
)

print("✅ Configuration d'entraînement prête !")

# %%
# Étape 9 : Entraînement
print("🚀 DÉBUT DE L'ENTRAÎNEMENT...")
print("⏱️  Durée estimée : 15-30 minutes")
print("-" * 50)

# Créer le Trainer avec le data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,  # Important pour le padding
)

# Démarrer l'entraînement
try:
    training_results = trainer.train()

    print("🎉 ENTRAÎNEMENT TERMINÉ !")
    print(f"📈 Perte finale : {training_results.metrics['train_loss']:.4f}")

    # Sauvegarder le modèle
    print("💾 Sauvegarde du modèle...")
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)
    print(f"✅ Modèle sauvegardé dans : {output_dir}")

except Exception as e:
    print(f"❌ Erreur lors de l'entraînement : {e}")
    print("\n🔧 Solutions possibles :")
    print("1. Assurez-vous d'avoir assez de mémoire GPU")
    print("2. Essayez de réduire max_length à 128")
    print("3. Réduisez le nombre d'exemples d'entraînement")
    print("4. Utilisez un Google Colab Pro avec plus de mémoire")

⚙️ Configuration du modèle...
🎯 Modèle sélectionné : unsloth/llama-3.2-1b
📥 Chargement du tokenizer...
✅ Tokenizer chargé : PreTrainedTokenizerFast
📥 Chargement du modèle (peut prendre 1-2 minutes)...
✅ Modèle chargé avec succès !
🔧 Préparation du modèle pour l'entraînement 4-bit...
✅ Modèle préparé pour l'entraînement 4-bit
⚙️ Configuration LoRA...
🔄 Application de LoRA au modèle...
trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
✅ Configuration LoRA terminée !
📊 Préparation des données...
🔠 Tokenisation des données...


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

/tmp/ipython-input-4210662833.py:165: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


✅ Données tokenisées !
   Train: 160 exemples
   Test: 40 exemples
⚙️ Configuration de l'entraînement...
✅ Configuration d'entraînement prête !
🚀 DÉBUT DE L'ENTRAÎNEMENT...
⏱️  Durée estimée : 15-30 minutes
--------------------------------------------------


Step,Training Loss,Validation Loss
50,0.336400,0.271106


🎉 ENTRAÎNEMENT TERMINÉ !
📈 Perte finale : 1.2630
💾 Sauvegarde du modèle...
✅ Modèle sauvegardé dans : ./emsi-llama-lora-model


In [21]:
# %%
# Étape 10 : Sauvegarde du modèle fine-tuné
print("💾 Sauvegarde du modèle...")

# Créer les dossiers de sauvegarde
lora_dir = "./emsi-lora-adapter"
full_model_dir = "./emsi-llama-finetuned"

import os
os.makedirs(lora_dir, exist_ok=True)
os.makedirs(full_model_dir, exist_ok=True)

# OPTION 1: Sauvegarder seulement l'adapter LoRA (léger)
model.save_pretrained(lora_dir)
tokenizer.save_pretrained(lora_dir)
print(f"✅ Adapter LoRA sauvegardé dans : {lora_dir}/")
print(f"   Taille : ~{os.path.getsize(lora_dir + '/adapter_model.bin') / 1024 / 1024:.1f} MB")

# OPTION 2: Sauvegarder le modèle complet (plus lourd)
# Note: Cela sauvegarde le modèle de base + LoRA
trainer.save_model(full_model_dir)
print(f"✅ Modèle complet sauvegardé dans : {full_model_dir}/")

print("\n📁 Structure des fichiers sauvegardés :")
print("1. emsi-lora-adapter/ - Adapter LoRA seulement (recommandé)")
print("   - adapter_config.json")
print("   - adapter_model.bin")
print("   - tokenizer files")
print("\n2. emsi-llama-finetuned/ - Modèle complet")
print("   - pytorch_model.bin")
print("   - config.json")
print("   - tokenizer files")

# OPTION 3: Sauvegarder au format safetensors (plus sûr)
try:
    model.save_pretrained(lora_dir, safe_serialization=True)
    print("\n🔒 Version safetensors également sauvegardée")
except:
    print("\n⚠️  Safetensors non disponible, format PyTorch utilisé")

# Sauvegarder aussi la configuration d'entraînement
import json
training_config = {
    "model_name": model_name,
    "lora_config": lora_config.__dict__,
    "training_args": training_args.to_dict(),
}
with open(f"{lora_dir}/training_config.json", "w") as f:
    json.dump(training_config, f, indent=2)

print("\n🎉 Toutes les sauvegardes sont terminées !")

💾 Sauvegarde du modèle...
✅ Adapter LoRA sauvegardé dans : ./emsi-lora-adapter/


FileNotFoundError: [Errno 2] No such file or directory: './emsi-lora-adapter/adapter_model.bin'

In [22]:
# %%
# Étape 10 : Vérification et organisation de la sauvegarde
print("🔍 VÉRIFICATION DE LA SAUVEGARDE...")

import os
import glob

# Chemin où le modèle a été sauvegardé (d'après votre message)
saved_path = "./emsi-llama-lora-model"

if os.path.exists(saved_path):
    print(f"✅ Modèle trouvé dans : {saved_path}")

    # Lister tous les fichiers sauvegardés
    files = os.listdir(saved_path)
    print(f"📁 Fichiers sauvegardés ({len(files)} fichiers):")

    for file in sorted(files):
        file_path = os.path.join(saved_path, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / 1024 / 1024
            print(f"   • {file} ({size_mb:.1f} MB)")
        else:
            print(f"   • {file}/ (dossier)")

    # Vérifier les fichiers importants
    important_files = [
        "pytorch_model.bin",
        "adapter_model.bin",
        "adapter_model.safetensors",
        "config.json",
        "tokenizer_config.json"
    ]

    print("\n🔍 Fichiers clés trouvés :")
    for imp_file in important_files:
        if imp_file in files:
            print(f"   ✓ {imp_file}")
        else:
            # Chercher des variations
            found = False
            for f in files:
                if imp_file.replace(".bin", "") in f or imp_file.replace(".json", "") in f:
                    print(f"   → {f} (variante de {imp_file})")
                    found = True
                    break
            if not found:
                print(f"   ✗ {imp_file} (non trouvé)")

    # Calculer la taille totale
    total_size = sum(os.path.getsize(os.path.join(saved_path, f))
                     for f in files if os.path.isfile(os.path.join(saved_path, f)))
    print(f"\n📊 Taille totale : {total_size / 1024 / 1024:.1f} MB")

    # Créer une copie organisée (optionnel)
    print("\n🔄 Création d'une copie organisée...")
    organized_path = "./emsi-llama-finetuned-organized"
    os.makedirs(organized_path, exist_ok=True)

    # Si c'est un adapter LoRA, renommer pour plus de clarté
    if "adapter_config.json" in files:
        print("   Type : Adapter LoRA détecté")
        # Créer un README
        with open(os.path.join(organized_path, "README.md"), "w") as f:
            f.write(f"""# Modèle Llama 3.2 Fine-tuné avec LoRA

## Spécifications
- Modèle de base : {model_name if 'model_name' in locals() else 'llama3.2'}
- Méthode : LoRA (Low-Rank Adaptation)
- Perte finale : 1.2630
- Date d'entraînement : {os.path.getctime(saved_path):%Y-%m-%d}

## Utilisation
```python
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Charger le modèle de base
base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3.2-1b")

# Charger l'adapter LoRA
model = PeftModel.from_pretrained(base_model, "{saved_path}")
tokenizer = AutoTokenizer.from_pretrained("{saved_path}")

SyntaxError: incomplete input (ipython-input-1878047746.py, line 65)

In [23]:
# %%
# Étape 10 : Vérification et organisation de la sauvegarde
print("🔍 VÉRIFICATION DE LA SAUVEGARDE...")

import os
import glob

# Chemin où le modèle a été sauvegardé (d'après votre message)
saved_path = "./emsi-llama-lora-model"

if os.path.exists(saved_path):
    print(f"✅ Modèle trouvé dans : {saved_path}")

    # Lister tous les fichiers sauvegardés
    files = os.listdir(saved_path)
    print(f"📁 Fichiers sauvegardés ({len(files)} fichiers):")

    for file in sorted(files):
        file_path = os.path.join(saved_path, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / 1024 / 1024
            print(f"   • {file} ({size_mb:.1f} MB)")
        else:
            print(f"   • {file}/ (dossier)")

    # Vérifier les fichiers importants
    important_files = [
        "pytorch_model.bin",
        "adapter_model.bin",
        "adapter_model.safetensors",
        "config.json",
        "tokenizer_config.json"
    ]

    print("\n🔍 Fichiers clés trouvés :")
    for imp_file in important_files:
        if imp_file in files:
            print(f"   ✓ {imp_file}")
        else:
            # Chercher des variations
            found = False
            for f in files:
                if imp_file.replace(".bin", "") in f or imp_file.replace(".json", "") in f:
                    print(f"   → {f} (variante de {imp_file})")
                    found = True
                    break
            if not found:
                print(f"   ✗ {imp_file} (non trouvé)")

    # Calculer la taille totale
    total_size = sum(os.path.getsize(os.path.join(saved_path, f))
                     for f in files if os.path.isfile(os.path.join(saved_path, f)))
    print(f"\n📊 Taille totale : {total_size / 1024 / 1024:.1f} MB")

    # Créer une copie organisée (optionnel)
    print("\n🔄 Création d'une copie organisée...")
    organized_path = "./emsi-llama-finetuned-organized"
    os.makedirs(organized_path, exist_ok=True)

    # Si c'est un adapter LoRA, renommer pour plus de clarté
    if "adapter_config.json" in files:
        print("   Type : Adapter LoRA détecté")
        # Créer un README
        with open(os.path.join(organized_path, "README.md"), "w") as f:
            f.write(f"""# Modèle Llama 3.2 Fine-tuné avec LoRA

## Spécifications
- Modèle de base : {model_name if 'model_name' in locals() else 'llama3.2'}
- Méthode : LoRA (Low-Rank Adaptation)
- Perte finale : 1.2630
- Date d'entraînement : {os.path.getctime(saved_path):%Y-%m-%d}

## Utilisation
```python
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Charger le modèle de base
base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3.2-1b")

# Charger l'adapter LoRA
model = PeftModel.from_pretrained(base_model, "{saved_path}")
tokenizer = AutoTokenizer.from_pretrained("{saved_path}")

SyntaxError: incomplete input (ipython-input-1878047746.py, line 65)

In [24]:
# %%
# Étape 10 : Vérification et organisation de la sauvegarde
print("🔍 VÉRIFICATION DE LA SAUVEGARDE...")

import os
import glob

# Chemin où le modèle a été sauvegardé (d'après votre message)
saved_path = "./emsi-llama-lora-model"

if os.path.exists(saved_path):
    print(f"✅ Modèle trouvé dans : {saved_path}")

    # Lister tous les fichiers sauvegardés
    files = os.listdir(saved_path)
    print(f"📁 Fichiers sauvegardés ({len(files)} fichiers):")

    for file in sorted(files):
        file_path = os.path.join(saved_path, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / 1024 / 1024
            print(f"   • {file} ({size_mb:.1f} MB)")
        else:
            print(f"   • {file}/ (dossier)")

    # Vérifier les fichiers importants
    important_files = [
        "pytorch_model.bin",
        "adapter_model.bin",
        "adapter_model.safetensors",
        "config.json",
        "tokenizer_config.json"
    ]

    print("\n🔍 Fichiers clés trouvés :")
    for imp_file in important_files:
        if imp_file in files:
            print(f"   ✓ {imp_file}")
        else:
            # Chercher des variations
            found = False
            for f in files:
                if imp_file.replace(".bin", "") in f or imp_file.replace(".json", "") in f:
                    print(f"   → {f} (variante de {imp_file})")
                    found = True
                    break
            if not found:
                print(f"   ✗ {imp_file} (non trouvé)")

    # Calculer la taille totale
    total_size = 0
    for f in files:
        file_path = os.path.join(saved_path, f)
        if os.path.isfile(file_path):
            total_size += os.path.getsize(file_path)
    print(f"\n📊 Taille totale : {total_size / 1024 / 1024:.1f} MB")

    # Créer une copie organisée (optionnel)
    print("\n🔄 Création d'une copie organisée...")
    organized_path = "./emsi-llama-finetuned-organized"
    os.makedirs(organized_path, exist_ok=True)

    # Si c'est un adapter LoRA, renommer pour plus de clarté
    if "adapter_config.json" in files:
        print("   Type : Adapter LoRA détecté")
        # Créer un README plus simple
        readme_content = f"""# Modèle Llama 3.2 Fine-tuné avec LoRA

## Spécifications
- Modèle de base : {"llama3.2-1b" if 'MODEL_CHOICE' in locals() else 'llama3.2'}
- Méthode : LoRA (Low-Rank Adaptation)
- Perte finale : 1.2630
- Date : {os.path.getctime(saved_path):%Y-%m-%d}

## Utilisation
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# Charger le modèle de base
base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3.2-1b")

# Charger l'adapter LoRA
model = PeftModel.from_pretrained(base_model, "{saved_path}")
tokenizer = AutoTokenizer.from_pretrained("{saved_path}")

## Fichiers
- adapter_model.bin : Poids LoRA
- adapter_config.json : Configuration LoRA
- tokenizer* : Fichiers du tokenizer
"""

        with open(os.path.join(organized_path, "README.md"), "w") as f:
            f.write(readme_content)

    # Copier les fichiers
    import shutil
    try:
        for file in files:
            src = os.path.join(saved_path, file)
            dst = os.path.join(organized_path, file)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
        print(f"   ✅ Copie organisée créée dans : {organized_path}")
    except Exception as e:
        print(f"   ⚠️  Erreur lors de la copie : {e}")

else:
    print(f"❌ Le chemin {saved_path} n'existe pas !")
    print("Tentative de recherche d'autres sauvegardes...")

    # Chercher d'autres sauvegardes possibles
    possible_paths = [
        "./emsi-llama-lora-model",
        "./emsi-lora-adapter",
        "./output",
        "./results",
        "./checkpoint-*"
    ]

    found_any = False
    for path_pattern in possible_paths:
        matches = glob.glob(path_pattern)
        for match in matches:
            if os.path.isdir(match):
                print(f"   Trouvé : {match}")
                match_files = os.listdir(match)[:5]  # Premiers 5 fichiers
                print(f"     Fichiers : {', '.join(match_files)}")
                found_any = True

    if not found_any:
        print("   Aucune sauvegarde trouvée")

print("\n" + "="*50)
print("🎉 FINE-TUNING TERMINÉ AVEC SUCCÈS !")
print("="*50)
print("\n📋 Prochaines étapes :")
print("1. Tester votre modèle avec quelques exemples")
print("2. Évaluer les performances sur votre tâche")
print("3. Partager le modèle si nécessaire")

# Option : Tester rapidement le modèle
print("\n🧪 Pour tester rapidement votre modèle :")
print(f"""
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Charger
base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3.2-1b",
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, "{saved_path}")
tokenizer = AutoTokenizer.from_pretrained("{saved_path}")

# Tester
input_text = "Votre prompt ici"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
""")

🔍 VÉRIFICATION DE LA SAUVEGARDE...
✅ Modèle trouvé dans : ./emsi-llama-lora-model
📁 Fichiers sauvegardés (8 fichiers):
   • README.md (0.0 MB)
   • adapter_config.json (0.0 MB)
   • adapter_model.safetensors (43.0 MB)
   • checkpoint-60/ (dossier)
   • special_tokens_map.json (0.0 MB)
   • tokenizer.json (16.4 MB)
   • tokenizer_config.json (0.0 MB)
   • training_args.bin (0.0 MB)

🔍 Fichiers clés trouvés :
   ✗ pytorch_model.bin (non trouvé)
   → adapter_model.safetensors (variante de adapter_model.bin)
   ✓ adapter_model.safetensors
   → adapter_config.json (variante de config.json)
   ✓ tokenizer_config.json

📊 Taille totale : 59.5 MB

🔄 Création d'une copie organisée...
   Type : Adapter LoRA détecté


ValueError: Invalid format specifier '%Y-%m-%d' for object of type 'float'

In [25]:
# %%
# Étape 10 : Vérification et organisation de la sauvegarde
print("🔍 VÉRIFICATION DE LA SAUVEGARDE...")

import os
import glob
from datetime import datetime

# Chemin où le modèle a été sauvegardé
saved_path = "./emsi-llama-lora-model"

if os.path.exists(saved_path):
    print(f"✅ Modèle trouvé dans : {saved_path}")

    # Lister tous les fichiers sauvegardés
    files = os.listdir(saved_path)
    print(f"📁 Fichiers sauvegardés ({len(files)} fichiers):")

    for file in sorted(files):
        file_path = os.path.join(saved_path, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / 1024 / 1024
            print(f"   • {file} ({size_mb:.1f} MB)")
        else:
            print(f"   • {file}/ (dossier)")

    # Vérifier les fichiers importants
    important_files = [
        "pytorch_model.bin",
        "adapter_model.bin",
        "adapter_model.safetensors",
        "config.json",
        "tokenizer_config.json"
    ]

    print("\n🔍 Fichiers clés trouvés :")
    for imp_file in important_files:
        if imp_file in files:
            print(f"   ✓ {imp_file}")
        else:
            # Chercher des variations
            found = False
            for f in files:
                if imp_file.replace(".bin", "") in f or imp_file.replace(".json", "") in f:
                    print(f"   → {f} (variante de {imp_file})")
                    found = True
                    break
            if not found:
                print(f"   ✗ {imp_file} (non trouvé)")

    # Calculer la taille totale
    total_size = 0
    for f in files:
        file_path = os.path.join(saved_path, f)
        if os.path.isfile(file_path):
            total_size += os.path.getsize(file_path)
    print(f"\n📊 Taille totale : {total_size / 1024 / 1024:.1f} MB")

    # Obtenir la date de création
    try:
        timestamp = os.path.getctime(saved_path)
        date_str = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d')
    except:
        date_str = datetime.now().strftime('%Y-%m-%d')

    # Créer une copie organisée (optionnel)
    print("\n🔄 Création d'une copie organisée...")
    organized_path = "./emsi-llama-finetuned-organized"
    os.makedirs(organized_path, exist_ok=True)

    # Si c'est un adapter LoRA, renommer pour plus de clarté
    if "adapter_config.json" in files:
        print("   Type : Adapter LoRA détecté")
        # Créer un README plus simple
        readme_content = f"""# Modèle Llama 3.2 Fine-tuné avec LoRA

## Spécifications
- Modèle de base : {"llama3.2-1b" if 'MODEL_CHOICE' in locals() else 'llama3.2'}
- Méthode : LoRA (Low-Rank Adaptation)
- Perte finale : 1.2630
- Date : {date_str}

## Utilisation
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# Charger le modèle de base
base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3.2-1b")

# Charger l'adapter LoRA
model = PeftModel.from_pretrained(base_model, "{saved_path}")
tokenizer = AutoTokenizer.from_pretrained("{saved_path}")

## Fichiers
- adapter_model.bin : Poids LoRA
- adapter_config.json : Configuration LoRA
- tokenizer* : Fichiers du tokenizer
"""

        with open(os.path.join(organized_path, "README.md"), "w") as f:
            f.write(readme_content)

    # Copier les fichiers
    import shutil
    try:
        for file in files:
            src = os.path.join(saved_path, file)
            dst = os.path.join(organized_path, file)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
        print(f"   ✅ Copie organisée créée dans : {organized_path}")
    except Exception as e:
        print(f"   ⚠️  Erreur lors de la copie : {e}")

else:
    print(f"❌ Le chemin {saved_path} n'existe pas !")
    print("Tentative de recherche d'autres sauvegardes...")

    # Chercher d'autres sauvegardes possibles
    possible_paths = [
        "./emsi-llama-lora-model",
        "./emsi-lora-adapter",
        "./output",
        "./results",
        "./checkpoint-*"
    ]

    found_any = False
    for path_pattern in possible_paths:
        matches = glob.glob(path_pattern)
        for match in matches:
            if os.path.isdir(match):
                print(f"   Trouvé : {match}")
                match_files = os.listdir(match)[:5]
                print(f"     Fichiers : {', '.join(match_files)}")
                found_any = True

    if not found_any:
        print("   Aucune sauvegarde trouvée")

print("\n" + "="*50)
print("🎉 FINE-TUNING TERMINÉ AVEC SUCCÈS !")
print("="*50)
print(f"\n📊 Résultat : Perte finale = 1.2630")
print("✅ Votre modèle est prêt à être utilisé !")

print("\n🧪 Code pour tester votre modèle :")
print(f"""
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Charger le modèle
print("Chargement du modèle...")
base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3.2-1b",
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, "{saved_path}")
tokenizer = AutoTokenizer.from_pretrained("{saved_path}")

# 2. Tester
print("\\nTest du modèle...")
input_text = "Question : Quel est le capital de la France ?\\nRéponse :"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Réponse : {{response}}")
""")

🔍 VÉRIFICATION DE LA SAUVEGARDE...
✅ Modèle trouvé dans : ./emsi-llama-lora-model
📁 Fichiers sauvegardés (8 fichiers):
   • README.md (0.0 MB)
   • adapter_config.json (0.0 MB)
   • adapter_model.safetensors (43.0 MB)
   • checkpoint-60/ (dossier)
   • special_tokens_map.json (0.0 MB)
   • tokenizer.json (16.4 MB)
   • tokenizer_config.json (0.0 MB)
   • training_args.bin (0.0 MB)

🔍 Fichiers clés trouvés :
   ✗ pytorch_model.bin (non trouvé)
   → adapter_model.safetensors (variante de adapter_model.bin)
   ✓ adapter_model.safetensors
   → adapter_config.json (variante de config.json)
   ✓ tokenizer_config.json

📊 Taille totale : 59.5 MB

🔄 Création d'une copie organisée...
   Type : Adapter LoRA détecté
   ✅ Copie organisée créée dans : ./emsi-llama-finetuned-organized

🎉 FINE-TUNING TERMINÉ AVEC SUCCÈS !

📊 Résultat : Perte finale = 1.2630
✅ Votre modèle est prêt à être utilisé !

🧪 Code pour tester votre modèle :

from peft import PeftModel
from transformers import AutoModelForCausa

In [26]:
# %%
# Étape 11 : Test du modèle fine-tuné sur EMSI
print("🧪 TEST DU MODÈLE FINE-TUNÉ EMSI")
print("=" * 50)

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import textwrap

# Chemin du modèle
model_path = "./emsi-llama-lora-model"  # Chemin où votre modèle est sauvegardé
base_model_name = "unsloth/llama-3.2-1b"  # Modèle de base utilisé

print(f"📂 Chemin du modèle : {model_path}")
print(f"🤖 Modèle de base : {base_model_name}")

try:
    # 1. Charger le tokenizer
    print("\n🔤 Chargement du tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Tokenizer chargé")

    # 2. Charger le modèle de base
    print("📥 Chargement du modèle de base...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        trust_remote_code=True
    )
    print("✅ Modèle de base chargé")

    # 3. Charger l'adapter LoRA
    print("🔗 Chargement de l'adapter LoRA...")
    model = PeftModel.from_pretrained(base_model, model_path)
    model.eval()
    print("✅ Adapter LoRA chargé")

    # 4. Créer le pipeline
    print("⚙️ Création du pipeline...")
    chatbot = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto",
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1
    )

    print("🎉 Modèle EMSI fine-tuné chargé avec succès !")

except Exception as e:
    print(f"❌ Erreur lors du chargement : {e}")
    print("\n⚠️  Solutions possibles :")
    print("1. Vérifiez que le chemin du modèle est correct")
    print("2. Essayez de recharger le notebook")
    print("3. Vérifiez que vous avez assez de mémoire GPU")
    raise

# %%
# Questions de test spécifiques à EMSI
print("\n📝 TESTS AUTOMATIQUES EMSI :")
print("=" * 50)

test_questions = [
    "Quand a été fondée l'EMSI ?",
    "Combien de campus possède l'EMSI ?",
    "Le diplôme EMSI est-il reconnu par l'État ?",
    "Quelles sont les spécialités disponibles à l'EMSI ?",
    "Quelle est la durée des formations à l'EMSI ?",
    "L'EMSI propose-t-elle des formations en alternance ?",
    "Quels sont les frais de scolarité à l'EMSI ?",
    "Qui est le fondateur de l'EMSI ?",
    "L'EMSI a-t-elle des partenariats internationaux ?",
    "Comment contacter l'EMSI ?"
]

print(f"🔢 {len(test_questions)} questions de test préparées")

for i, question in enumerate(test_questions, 1):
    # Format du prompt (identique à celui utilisé pendant l'entraînement)
    prompt = f"### Question: {question}\n\n### Réponse:"

    print(f"\n{'='*60}")
    print(f"❓ Question {i}/{len(test_questions)}: {question}")
    print(f"{'-'*60}")

    try:
        # Générer la réponse
        start_time = torch.cuda.Event(enable_timing=True)
        end_time = torch.cuda.Event(enable_timing=True)

        start_time.record()
        response = chatbot(prompt, max_new_tokens=150)[0]['generated_text']
        end_time.record()
        torch.cuda.synchronize()

        # Extraire seulement la réponse
        if "### Réponse:" in response:
            answer = response.split("### Réponse:")[1].strip()
        else:
            answer = response.replace(prompt, "").strip()

        # Calculer le temps
        elapsed_time = start_time.elapsed_time(end_time) / 1000.0

        # Afficher proprement
        print(f"💡 Réponse ({elapsed_time:.2f}s):")
        wrapped_answer = textwrap.fill(answer, width=70)
        print(f"{wrapped_answer}")

        # Évaluer la pertinence (simple)
        keywords = question.lower().split()
        relevant_keywords = sum(1 for keyword in keywords if keyword in answer.lower())
        relevance_score = min(10, relevant_keywords * 2)
        print(f"📊 Pertinence estimée: {relevance_score}/10")

    except Exception as e:
        print(f"❌ Erreur : {e}")

    print()

# %%
# Test de compréhension contextuelle
print("\n🧠 TEST DE COMPRÉHENSION CONTEXTUELLE EMSI")
print("=" * 50)

context_tests = [
    {
        "context": "L'École Marocaine des Sciences de l'Ingénieur (EMSI) a été fondée en 1986 par M. Mohamed Kabbaj.",
        "question": "Qui a fondé l'EMSI ?"
    },
    {
        "context": "L'EMSI possède 12 campus au Maroc, dont les principaux sont à Casablanca, Rabat et Marrakech.",
        "question": "Combien de campus l'EMSI possède-t-elle ?"
    },
    {
        "context": "Les diplômes de l'EMSI sont reconnus par l'État marocain et accrédités par le ministère de l'Enseignement Supérieur.",
        "question": "Le diplôme EMSI est-il reconnu ?"
    }
]

for test in context_tests:
    prompt = f"### Contexte: {test['context']}\n\n### Question: {test['question']}\n\n### Réponse:"

    print(f"\n📚 Contexte: {test['context']}")
    print(f"❓ Question: {test['question']}")
    print(f"{'-'*50}")

    try:
        response = chatbot(prompt, max_new_tokens=100)[0]['generated_text']

        if "### Réponse:" in response:
            answer = response.split("### Réponse:")[1].strip()
        else:
            answer = response

        print(f"💡 Réponse: {answer}")

        # Vérifier si la réponse est cohérente avec le contexte
        context_lower = test['context'].lower()
        answer_lower = answer.lower()
        if any(keyword in answer_lower for keyword in test['context'].split()[:5]):
            print("✅ Cohérence avec le contexte: OUI")
        else:
            print("⚠️  Cohérence avec le contexte: À vérifier")

    except Exception as e:
        print(f"❌ Erreur : {e}")

    print()

# %%
# Test interactif
print("\n🎮 MODE INTERACTIF - Chat avec l'Assistant EMSI")
print("=" * 50)
print("💡 Tapez 'quit', 'exit' ou 'q' pour quitter")
print("💡 Tapez 'clear' pour effacer l'historique")
print("💡 Tapez 'help' pour voir les commandes disponibles")
print("-" * 50)

conversation_history = []

while True:
    user_input = input("\n👤 Vous: ").strip()

    if user_input.lower() in ['quit', 'exit', 'q']:
        print("👋 Au revoir ! Merci d'avoir testé l'Assistant EMSI.")
        break

    if user_input.lower() == 'clear':
        conversation_history = []
        print("🧹 Historique effacé.")
        continue

    if user_input.lower() == 'help':
        print("\n📋 Commandes disponibles:")
        print("  - quit/exit/q : Quitter")
        print("  - clear : Effacer l'historique")
        print("  - history : Voir l'historique")
        print("  - help : Afficher cette aide")
        continue

    if user_input.lower() == 'history':
        print("\n📜 Historique de la conversation:")
        for i, (q, a) in enumerate(conversation_history[-5:], 1):
            print(f"  {i}. Q: {q[:50]}...")
            print(f"     A: {a[:50]}...")
        continue

    # Construire le prompt avec historique
    if conversation_history:
        history_text = "\n".join([f"Q: {q}\nA: {a}" for q, a in conversation_history[-3:]])
        prompt = f"### Historique:\n{history_text}\n\n### Nouvelle Question: {user_input}\n\n### Réponse:"
    else:
        prompt = f"### Question: {user_input}\n\n### Réponse:"

    try:
        print("🤖 Assistant EMSI: ", end="", flush=True)

        # Générer la réponse avec stream
        response = chatbot(prompt, max_new_tokens=200)[0]['generated_text']

        if "### Réponse:" in response:
            answer = response.split("### Réponse:")[1].strip()
        else:
            answer = response

        print(f"{answer}")

        # Sauvegarder dans l'historique
        conversation_history.append((user_input, answer))

    except Exception as e:
        print(f"❌ Désolé, une erreur est survenue: {e}")

# %%
# Évaluation finale
print("\n📊 ÉVALUATION FINALE DU MODÈLE EMSI")
print("=" * 50)

if 'conversation_history' in locals() and conversation_history:
    print(f"📈 Nombre d'échanges : {len(conversation_history)}")

    # Analyser les réponses
    total_length = 0
    for _, answer in conversation_history:
        total_length += len(answer.split())

    if conversation_history:
        avg_length = total_length / len(conversation_history)
        print(f"📏 Longueur moyenne des réponses : {avg_length:.1f} mots")

    print("\n💬 Derniers échanges :")
    for i, (q, a) in enumerate(conversation_history[-3:], 1):
        print(f"\n  Échange {i}:")
        print(f"  Q: {q[:60]}..." if len(q) > 60 else f"  Q: {q}")
        print(f"  A: {a[:60]}..." if len(a) > 60 else f"  A: {a}")

print("\n" + "="*50)
print("✅ TEST DU MODÈLE EMSI TERMINÉ !")
print("="*50)

🧪 TEST DU MODÈLE FINE-TUNÉ EMSI
📂 Chemin du modèle : ./emsi-llama-lora-model
🤖 Modèle de base : unsloth/llama-3.2-1b

🔤 Chargement du tokenizer...
✅ Tokenizer chargé
📥 Chargement du modèle de base...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Modèle de base chargé
🔗 Chargement de l'adapter LoRA...


Device set to use cuda:0


✅ Adapter LoRA chargé
⚙️ Création du pipeline...
🎉 Modèle EMSI fine-tuné chargé avec succès !

📝 TESTS AUTOMATIQUES EMSI :
🔢 10 questions de test préparées

❓ Question 1/10: Quand a été fondée l'EMSI ?
------------------------------------------------------------
💡 Réponse (7.81s):
La École Marocaine des Sciences de l'Ingénieur (EMSI) a été fondée en
1986.  ## Contexte:  L'histoire de l'EMSI : une découverte importante
## Contexte de la question:  Quand a été fondée l'EMSI?  ## Réponse
détaillée:  **D'après les sources :** L'EMSI est un établissement
public marocain fondé en 1986. Sa date d'inscription officielle dans
la répartition des formations académiques passe par le décret no
2005-27 du 30 janvier 2005. Le décret précise que l'EMSI couvre les
📊 Pertinence estimée: 10/10


❓ Question 2/10: Combien de campus possède l'EMSI ?
------------------------------------------------------------
💡 Réponse (4.35s):
L'EMSI dispose de plus de 10 campus dans des villes différentes :
Casablanca, Ra

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


💡 Réponse (6.47s):
Pour les questions concernant l'emsi : le service client EMSI est
accessible sur demande au 04 72 46 10 00 ou via email à
[email protected]. pour les réponses détaillées veuillez consulter la
page web officielle de l'emsi : https://www.emsi.fr/fr/contact/  ###
Contexte:  Département des relations publiques de l'emsi - contactez-
nous  ### Contexte détaillé:  Pour les questions concernant l'emsi :
le service client emsi est accessible sur demande au 04 72 46 10 00 ou
via email à [email protected]. pour les réponses détaillées veu
📊 Pertinence estimée: 2/10


🧠 TEST DE COMPRÉHENSION CONTEXTUELLE EMSI

📚 Contexte: L'École Marocaine des Sciences de l'Ingénieur (EMSI) a été fondée en 1986 par M. Mohamed Kabbaj.
❓ Question: Qui a fondé l'EMSI ?
--------------------------------------------------
💡 Réponse: L'EMSI a été fondée en 1986 par :

**Monsieur Mohamed Kabbaj**, un éminent enseignant marocain, débute sa carrière comme professeur au CIDEA de Casablanca en 1967. Il pas

In [27]:
# %%
# TÉLÉCHARGEMENT DU MODÈLE
print("💾 TÉLÉCHARGEMENT DU MODÈLE FINE-TUNÉ")
print("="*50)

import os
import shutil
from google.colab import files

# Chemin de votre modèle
model_path = "./emsi-llama-lora-model"
output_zip = "emsi-llama-model.zip"

# Vérifier que le modèle existe
if not os.path.exists(model_path):
    print(f"❌ Erreur : Le modèle n'existe pas dans {model_path}")
    print("📁 Dossiers disponibles :")
    for item in os.listdir("."):
        if os.path.isdir(item):
            print(f"  • {item}")
else:
    print(f"✅ Modèle trouvé : {model_path}")
    print(f"📊 Taille du dossier : {sum(os.path.getsize(os.path.join(model_path, f)) for f in os.listdir(model_path) if os.path.isfile(os.path.join(model_path, f))) / 1024 / 1024:.1f} MB")

    # Option 1 : Zipper et télécharger
    print("\n📦 Méthode 1 : Compression en ZIP")
    try:
        shutil.make_archive("emsi-llama-model", 'zip', model_path)
        print(f"✅ ZIP créé : emsi-llama-model.zip")

        # Télécharger
        print("⬇️  Téléchargement du fichier ZIP...")
        files.download("emsi-llama-model.zip")

    except Exception as e:
        print(f"❌ Erreur lors de la compression : {e}")

    # Option 2 : Télécharger fichier par fichier
    print("\n📁 Méthode 2 : Télécharger les fichiers individuellement")
    important_files = []
    for file in os.listdir(model_path):
        if file.endswith(('.bin', '.json', '.txt', '.md', '.safetensors')):
            important_files.append(file)

    print(f"📄 {len(important_files)} fichiers importants trouvés :")
    for file in important_files:
        print(f"  • {file}")

    # Télécharger les fichiers importants
    for file in important_files:
        try:
            shutil.copy(os.path.join(model_path, file), file)
            files.download(file)
            os.remove(file)  # Nettoyer
            print(f"  ✅ {file} téléchargé")
        except Exception as e:
            print(f"  ❌ Erreur pour {file}: {e}")

print("\n" + "="*50)
print("🎉 TÉLÉCHARGEMENT PRÊT !")
print("="*50)

💾 TÉLÉCHARGEMENT DU MODÈLE FINE-TUNÉ
✅ Modèle trouvé : ./emsi-llama-lora-model
📊 Taille du dossier : 59.5 MB

📦 Méthode 1 : Compression en ZIP
✅ ZIP créé : emsi-llama-model.zip
⬇️  Téléchargement du fichier ZIP...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📁 Méthode 2 : Télécharger les fichiers individuellement
📄 7 fichiers importants trouvés :
  • adapter_model.safetensors
  • special_tokens_map.json
  • adapter_config.json
  • tokenizer.json
  • training_args.bin
  • tokenizer_config.json
  • README.md


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

  ✅ tokenizer_config.json téléchargé


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ✅ README.md téléchargé

🎉 TÉLÉCHARGEMENT PRÊT !
